In [1]:
import kfp
import requests

USERNAME = "user@example.com"
PASSWORD = "12341234"
NAMESPACE = "kubeflow-user-example-com"
HOST = "<http://192.168.0.244:80>" # istio-ingressgateway's external-ip created by the load balancer.

session = requests.Session()
response = session.get(HOST)

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
}

data = {"login": USERNAME, "password": PASSWORD}
session.post(response.url, headers=headers, data=data)
session_cookie = session.cookies.get_dict()["authservice_session"]

client = kfp.Client(
    host=f"{HOST}/pipeline",
    namespace=f"{NAMESPACE}",
    cookies=f"authservice_session={session_cookie}",
)
print(client.list_pipelines())


InvalidSchema: No connection adapters were found for '<http://192.168.0.244:80>'

In [ ]:
# KF UI에 업로드된 파이프라인의 ID 찾기
pipeline_id = client.get_pipeline_id("automl_pipeline")

In [ ]:

# KF UI에 업로드된 파이프라인 정보 획득
info_pipeline = client.get_pipeline(pipeline_id=pipeline_id)
info_pipeline

In [ ]:

# 실행된 Run 정보를 확인합니다. 실행중이면 {'status': 'Running'},  종료되었으면 {'status': 'Succeeded'} 입니다.
info_run = client.get_run(run_id="34b2ee3a-1aa6-43cb-a2d8-6a1dab9e24cb")
info_run

In [ ]:

# 로컬 컨텍스트 구성에서 사용자 네임스페이스를 확인합니다. 지정되어있지 않으면 공백입니다.
client.get_user_namespace()

In [ ]:

# experiment 리스트를 확인합니다.
list_experiments = client.list_experiments()
print(list_experiments)
experiment_id = list_experiments.experiments[0].id
print(experiment_id)

In [ ]:

# 해당 id의 파이프라인 버전을 나열합니다.
list_pipeline_versions = client.list_pipeline_versions(pipeline_id=info_pipeline.default_version.id)
list_pipeline_versions

In [ ]:

# 모든 파이프라인 정보를 나열합니다.
list_pipelines = client.list_pipelines(page_size=50)
list_pipelines

In [ ]:

# 모든 파이프라인 이름만 나열합니다.
for i in range(list_pipelines.total_size):
    print(list_pipelines.pipelines[i].default_version.name)

In [ ]:

# experiment_id 내에 생성되어 있는 반복 작업 정보를 가져옵니다.
list_recurring_runs = client.list_recurring_runs(experiment_id=experiment_id)
list_recurring_runs
recurring_job_id = list_recurring_runs.jobs[0].id

In [ ]:

# recurring_run 세부 정보를 가져옵니다.
info_recurring_run = client.get_recurring_run(job_id=recurring_job_id)
info_recurring_run

In [ ]:

# 생성된 run을 나열하고, run id를 모두 가져옵니다.
list_runs = client.list_runs(page_size=50)
for i in range(list_runs.total_size):
    print(list_runs.runs[i].id)

In [ ]:

# kfp 상태체크
client.get_kfp_healthz()

In [ ]:

# KF UI 내 Experiments(KFP) 정보 획득
info_experiment = client.get_experiment(experiment_name="haram", namespace=NAMESPACE)
info_experiment

In [ ]:
# 파이프라인을 KF 클러스터에 업로드합니다.(UI에 등록), 같은 pipeline_name으로 중복 업로드는 불가능합니다.
client.upload_pipeline(pipeline_name="kfp_sdk_test", pipeline_package_path="./automl_pipeline.tar.gz", description="the pipeline uploaded by python sdk")

In [ ]:

# 파이프라인 버전을 업데이트합니다(UI에 등록)
updated_pipeline = client.upload_pipeline_version(pipeline_name="kfp_sdk_test", pipeline_version_name="v220614_2", pipeline_package_path="./automl_pipeline.tar.gz", description="the pipeline uploaded by python sdk")
updated_pipeline

In [ ]:

# 파이프라인을 실행해 run을 생성합니다. job_name=run_name이며, 컴파일된 파이프라인 파일의 경로를 지정합니다.
exec_run = client.run_pipeline(experiment_id=experiment_id, job_name="kfp_skd_test", pipeline_package_path="./automl_pipeline.tar.gz", enable_caching=False, params=None)
exec_run

In [ ]:

# Run이 완료될 때까지 기다렸다가 결과를 리턴합니다.
client.wait_for_run_completion(run_id=exec_run.id, timeout=345600)

In [ ]:
list_experiments = client.list_experiments()
experiment_id = list_experiments.experiments[0].id
client.upload_pipeline(pipeline_name="kfp_sdk_test", pipeline_package_path="./example_pipeline2.yaml", description="the pipeline uploaded by python sdk")
client.run_pipeline(experiment_id=experiment_id, job_name="kfp_skd_test", pipeline_package_path="./example_pipeline2.yaml", enable_caching=False, params=None)